In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [ ]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os
from tools import wait_for_node_finished
from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code, load_node

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [ ]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_opt',
                          computer_local,
                          '/home/max/Documents/alamode/bin/alm')
code_alm_opt = code
code = get_or_create_code('alamode.anphon',
                          computer_local,
                          '/home/max/Documents/alamode/bin/anphon')
code_anphon = code

code_alm_opt, code_anphon


In [ ]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


In [ ]:
from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [ ]:
g_cwd = nodebank.load("cwd")
g_dispfile_folder = nodebank.load("dispfile_folder")
g_displace_result = nodebank.load("displace_result")
g_superstructure = nodebank.load("superstructure")


# alm opt 

In [ ]:
g_alamodeextract_future = nodebank.load('harmic_extract')


In [ ]:
g_alamodeextract_future.outputs.dfset_file


In [ ]:
g_DFSET_file = g_alamodeextract_future.outputs.dfset_file
g_wd = nodebank.load("almprefix")  # subdirectory名
g_norder = nodebank.load("norder")


In [ ]:
g_prefix = nodebank.load('prefix')


In [ ]:
g_superstructure


In [ ]:
code = code_alm_opt

builder = code.get_builder()
builder.structure = g_superstructure  # 　安定構造
builder.prefix = g_prefix
builder.cwd = g_cwd
builder.norder = g_norder
builder.dfset_file = g_DFSET_file

g_almoptfuture = submit(builder)
print(g_almoptfuture)
wait_for_node_finished(g_almoptfuture)


In [ ]:
nodebank.dump("anphonfile_xml", g_almoptfuture.outputs.input_ANPHON_file)


# phonon dispersion

In [ ]:
g_root_wd = nodebank.load("root_wd")
g_root_wd  # 実行root directory


In [ ]:
g_wd  # subdirectory名


In [ ]:
g_property = Str(os.path.join(g_root_wd.value, "property")
                 )  # property directory
os.makedirs(g_property.value, exist_ok=True)
g_primstructure = nodebank.load("primstructure")  # primitive cell


In [ ]:
code = code_anphon

builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_wd
builder.cwd = g_property
# builder.norder = g_norder
builder.fcsxml = g_almoptfuture.outputs.input_ANPHON_file
builder.mode = Str("phonons")
builder.phonons_mode = Str("band")
# builder.kparam = Dict(dict={}) # should merge to the input port, param.

g_phbandfuture = submit(builder)
print(g_phbandfuture)
wait_for_node_finished(g_phbandfuture, 5)


In [ ]:
g_primstructure.get_ase()


In [ ]:
g_diag = nodebank.load("diag")
print(g_diag.get_array('diag'))


In [ ]:
# from alamode_aiida.alm_input import make_alm_uniform_kmesh
# g_qmesh_value = make_alm_uniform_kmesh(g_primstructure.get_ase(),kspacing=0.001)
g_qmesh = List(list=[10, 10, 10])  # qmeshの定義
g_qmesh.get_list()


In [ ]:
g_almprefix = "_".join([g_primstructure.get_formula(),
                        "x".join(map(str, g_diag.get_array("diag"))),
                        "harmonic",
                       "q"+"x".join(map(str, g_qmesh.get_list()))
                        ])
print(g_almprefix)

g_wd_dos = Str(g_almprefix)
g_wd_dos  # dos用の識別子


In [ ]:
#codename = "anphon@tutor"
#code = Code.get_from_string(codename)
code = code_anphon


builder = code.get_builder()
builder.structure = g_primstructure
builder.prefix = g_wd_dos
builder.cwd = g_property
# builder.norder = g_norder
builder.fcsxml = g_almoptfuture.outputs.input_ANPHON_file
builder.mode = Str("phonons")
builder.phonons_mode = Str("dos")
builder.qmesh = g_qmesh

g_phdosfuture = submit(builder)
print(g_phdosfuture)
wait_for_node_finished(g_phdosfuture, 5)


# plot dispersion

In [ ]:
g_wd


In [ ]:
phbandworkchain = WorkflowFactory('alamode.phband_img')
# g_img_filename = Str(f"{g_wd.value}_phband.pdf")
g_band_filenames = g_phbandfuture.outputs.phband_file
inputs = {"cwd": g_property, "band_filenames": g_band_filenames, "prefix": g_wd}
print(inputs)


In [ ]:

phbandimgworkchain = submit(phbandworkchain, **inputs)
print(phbandimgworkchain)
wait_for_node_finished(phbandimgworkchain)


In [ ]:
phbandimgworkchain.outputs.img_file.filename  # g_property directoryの出力ファイル


# plot DOS

In [ ]:

phdosworkchain = WorkflowFactory('alamode.phdos_img')

inputs = {"cwd": g_property, "dos_filenames": g_phdosfuture.outputs.phdos_file,
          "prefix": g_wd_dos}
print(inputs)

g_dosfuture = submit(phdosworkchain, **inputs)
print(g_dosfuture)
wait_for_node_finished(g_dosfuture)
g_dosfuture.outputs.img_file.filename


## free energies

In [ ]:
FreeenergyImgWorkChain = WorkflowFactory("alamode.freeenergy_img")
g_img_filename = Str("phdos_freeenergy.pdf")
g_inputs = {'cwd': g_property, 'prefix': g_wd_dos,
            'thermo_file': g_phdosfuture.outputs.thermo_file}

freeenergyimgfuture = nodebank.load("freeenergyimg", g_force)
if freeenergyimgfuture is None:
    # if True:
    freeenergyimgfuture = submit(FreeenergyImgWorkChain, **g_inputs)
    print(freeenergyimgfuture)
    wait_for_node_finished(freeenergyimgfuture)
    if freeenergyimgfuture.is_finished_ok:
        nodebank.dump("freeenergyimg", freeenergyimgfuture)


In [ ]:
freeenergyimgfuture.outputs.img_file.list_object_names()
